In [2]:
import pandas as pd
import numpy as np
import os
import re

### Data for Germany

In [3]:

# folder = r"C:\Users\jean-\Desktop\Cours ENSAE\3A ENSAE\ML for trading\Projet\ML-for-trading-project\Données projet\yield curve history Germany"

# # List all files in the folder
# files = os.listdir(folder)

# def extract_year(filename):
#     match = re.search(r'R(\d{2})XX', filename)
#     return match.group(1) if match else None

# # Create the dictionary of DataFrames with desired names
# yields_germany = {
#     f"yields Germany {extract_year(file)} years": pd.read_csv(os.path.join(folder, file),index_col =0)
#     for file in files if extract_year(file) is not None
# }

# list_years = [extract_year(file) for file in files if extract_year(file) is not None]

In [ ]:
# folder = r"./Données projet/yield curve history Germany"  # Use relative path for GitHub repo

# files = os.listdir(folder)

# def extract_year(filename):
#     match = re.search(r'R(\d{2})XX', filename)
#     return match.group(1) if match else None

# yields_germany = {
#     f"yields Germany {extract_year(file)} years": pd.read_csv(os.path.join(folder, file), index_col=0)
#     for file in files if extract_year(file) is not None
# }

# list_years = [extract_year(file) for file in files if extract_year(file) is not None]

In [ ]:

# start_date = '2000-01-01'
# end_date = '2025-10-13'

# business_days = pd.date_range(start=start_date, end=end_date, freq='B').strftime('%Y-%m-%d').tolist()

# for year in list_years:
#     df = yields_germany[f"yields Germany {year} years"]
#     series_filtered = df[df.index.isin(business_days)][f"BBSIS.D.I.ZST.ZI.EUR.S1311.B.A604.R{year}XX.R.A.A._Z._Z.A"]
#     df_filtered = pd.DataFrame(series_filtered)
#     df_filtered.columns = [f'{year}']
#     col_numeric = pd.to_numeric(df_filtered[f'{year}'], errors='coerce')
#     df_filtered = pd.DataFrame(col_numeric)
#     yields_germany[f"yields Germany {year} years"] = df_filtered[df_filtered[f'{year}'].notnull()]

    
   

### Data for US

For the US, we use the FRED MD database, available on the St Louis website, which contains a wide range of macroeconomic variables with monthly frequency.

In [10]:
df= pd.read_csv("./Données projet/data US/2025-09-MD.csv", index_col=0)

In [17]:
df

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
sasdate,,,,,,,,,,,,,,,,,,,,,
Transform:,5.000,5.0,5.000,5.000000e+00,5.00000,5.0000,5.0000,5.0000,5.0000,5.0000,...,6.000,6.000,6.00,6.00,6.00,2.0,6.00,6.00,6.0000,1.0000
1/1/1959,2583.560,2426.0,15.188,2.766768e+05,17689.23968,21.9616,23.3868,22.2620,31.6664,18.9498,...,18.294,10.152,2.13,2.45,2.04,NaN,6476.00,12298.00,84.2043,NaN
2/1/1959,2593.596,2434.8,15.346,2.787140e+05,17819.01912,22.3917,23.7024,22.4549,31.8987,19.0492,...,18.302,10.167,2.14,2.46,2.05,NaN,6476.00,12298.00,83.5280,NaN
3/1/1959,2610.396,2452.7,15.491,2.777753e+05,17967.91336,22.7142,23.8459,22.5651,31.8987,19.4223,...,18.289,10.185,2.15,2.45,2.07,NaN,6508.00,12349.00,81.6405,NaN
4/1/1959,2627.446,2470.0,15.435,2.833627e+05,17978.97983,23.1981,24.1903,22.8957,32.4019,19.5466,...,18.300,10.221,2.16,2.47,2.08,NaN,6620.00,12484.00,81.8099,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4/1/2025,20698.762,16739.9,123.748,1.555663e+06,721789.00000,103.6224,101.3671,101.1112,101.6979,101.0930,...,119.658,131.767,32.22,36.96,28.78,52.2,554180.13,940362.47,5500.0706,32.5116
5/1/2025,20581.743,16703.7,123.575,1.550522e+06,716101.00000,103.6570,101.4038,101.1856,101.5808,102.5239,...,119.780,132.071,32.31,37.08,28.87,52.2,551829.26,938763.49,5526.3170,20.3620
6/1/2025,20575.971,16664.7,123.894,1.556845e+06,723033.00000,104.2115,101.7271,101.5445,101.9628,101.0181,...,120.208,132.386,32.40,37.23,28.94,60.7,549682.41,937344.92,5555.2136,18.3246


In [12]:
df.describe()

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
count,801.000000,801.000000,801.000000,8.000000e+02,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,...,801.000000,801.000000,801.000000,801.000000,801.000000,647.000000,800.000000,800.000000,801.000000,759.000000
mean,9676.729325,8232.720100,57.303749,8.303029e+05,223252.860662,67.965425,72.135863,70.924318,79.337440,67.042342,...,63.716896,54.591798,12.931473,15.004444,11.914769,84.690572,142905.059450,359727.393038,1378.618822,19.278566
std,5178.645586,4100.614248,31.316873,3.904708e+05,192090.783435,27.032184,27.292419,27.570917,23.533362,28.789153,...,31.524712,35.020187,8.175654,9.183192,7.335325,13.659669,146268.035165,341935.326025,1538.347702,7.075780
min,5.000000,5.000000,5.000000,5.000000e+00,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,6.000000,6.000000,2.130000,2.450000,2.040000,2.000000,6.000000,6.000000,6.000000,1.000000
25%,5148.700000,4583.400000,29.110000,4.892787e+05,50436.918730,45.022100,47.154300,44.920000,61.377200,42.258000,...,33.313000,19.436000,5.170000,6.810000,4.780000,74.200000,10828.250000,34319.000000,192.315600,14.294950
50%,8487.783000,7313.200000,49.254000,7.239413e+05,159921.000000,62.919900,71.763800,69.964400,81.760700,61.828000,...,67.390000,52.481000,11.960000,13.760000,11.360000,88.600000,81295.500000,190899.875000,756.001700,17.621600
75%,13789.923000,11624.900000,84.409000,1.185876e+06,355792.000000,95.929400,99.454400,98.730900,100.669600,98.204900,...,93.328000,84.159000,19.710000,22.390000,17.990000,94.700000,269654.840000,731629.750000,2143.175800,22.513350
max,22918.422000,16739.900000,124.805000,1.565742e+06,732010.000000,104.211500,110.212000,108.675800,111.757500,109.816700,...,120.620000,133.209000,32.560000,37.470000,29.030000,112.000000,559708.930000,949306.220000,5846.707800,62.964800


In [18]:
pd.set_option('display.max_seq_items', None)  # pas de "..." pour les Index/Series
pd.set_option('display.max_columns', None)    # affiche toutes les colonnes
print(df.columns)

Index(['RPI', 'W875RX1', 'DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx', 'INDPRO',
       'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD', 'IPBUSEQ',
       'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S', 'IPFUELS',
       'CUMFNS', 'HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV', 'UNRATE', 'UEMPMEAN',
       'UEMPLT5', 'UEMP5TO14', 'UEMP15OV', 'UEMP15T26', 'UEMP27OV', 'CLAIMSx',
       'PAYEMS', 'USGOOD', 'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP',
       'NDMANEMP', 'SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE',
       'USGOVT', 'CES0600000007', 'AWOTMAN', 'AWHMAN', 'HOUST', 'HOUSTNE',
       'HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW',
       'PERMITS', 'PERMITW', 'ACOGNO', 'AMDMNOx', 'ANDENOx', 'AMDMUOx',
       'BUSINVx', 'ISRATIOx', 'M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS',
       'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'S&P 500',
       'S&P div yield', 'S&P PE ratio', 'FEDFUNDS', 'CP3Mx', 'TB3MS', 'TB6MS',
       'GS1', 'G

First, we keep only variables that may have an impact on the yield of the bonds and on interest rates, to ensure we don't overfit the data by training our models on irrelevant variables.

We don't take too much risk and only remove very irrelevant variables; For now many variables are probably very correlated and some should be removed later.

In [20]:
columns_to_keep = [#production and revenue variables
    'RPI',  #Real personal income; monitored by FED for interest rates decisions
                   'INDPRO', #Industrial production index; leading indicator of economic health, monitored by FED 
                   #job market variables
                   'HWI', #help wanted index, corresponds to the number of job offers
                   'UNRATE', #unemployment rate
                   'UEMP5TO14', #unemployment for people unemployed between 5 and 14 weeks = short term unemployment
                   'UEMP27OV', #unemployment for people unemployed for 27 weeks and over = long term unemployment
                   'CLAIMSx', #initial jobless claims, leading indicator of unemployment
                    'PAYEMS', #total non farm payroll, leading indicator of employment
                    'USGOOD',   #below is the breakdown of employment figures for various economic sectors
                    'CES1021000001',
                    'USCONS',
                    'MANEMP',
                    'DMANEMP',
                    'NDMANEMP',
                    'SRVPRD',
                    'USTPU',
                    'USWTRADE',
                    'USTRADE',
                    'USFIRE',
                    'USGOVT',
                    'NAPMEI', #another index on manufacturing employment
                    #consumption and housing
                    'HOUST', #housing variable, sensitive to interest rates and economic anticipations
                    'PERMIT', #same but for building permits
                    #order and stocks variables
                    ' DPCERA3M086SBEA', #real personal consumption expenditures
                    'CMRMTSPLX', #sales from industry
                    'RETAILX', #sales from retail
                    'NAPM', #indicator of activity, impacts growth anticipations
                    'NAPMNOI', #the 3 below variables are indicators of offer
                     'NAPMSDI', 
                     'NAPMII', 
                     'ACOGNO', #the 4 below are indicators of demand (orders in various sectors)
                     'AMDMNOX', 
                     'ANDENOX',
                    'AMDMUOX',
                    'ISRATIOX', #ratio stock over sales, gives an idea of supply demand dynamics
                    'UMCSENTX',  #imporant: consumer sentiment index, impact growth and consumption anticipations
                    #money and credit variables
                    'M1SL', #monetary base, important for interest rates and inflation anticipations
                    'M2SL', 
                    'M2REAL', #real monetary base
                    'AMBSL', #monetary base
                    'TOTRESNS', #total reserves of banks
                    'NONBORRES', #non borrowable reserves of banks
                    'BUSLOANS', #business loans, important for growth anticipations
                    'REALLN', #real loans, related to economic activity
                    'NONREVSL', #non-revolving consumer credit, related to consumption
                    'CONSPI', #consumer credit outstanding, related to consumption
                    'MZMSL', #monetary base
                    'DTCOLNVHFNM', #total nonfinancial corporate business liabilities, related to economic activity
                    'DTCTHFNM', #total household liabilities, related to consumption
                    'INVEST', #gross private domestic investment, related to economic activity
                    #interest rates variables
                    'FEDFUNDS', #federal funds rate, short term interest rates
                    'CP3Mx', 'TB3MS', 'TB6MS', 'GS1', 'GS5', 'GS10', #TO BE REMOVED WHEN YIELDS ARE ADDED yields of government treasuries for maturies <10y
                    'AAA', 'BAA', #yields of corporate bonds with notations AAA and BAA
                    'TWEXMMTH', 'EXSZUSx', 'EXJPUSx', 'EXUSUKx', 'EXCAUSx', #exchange rates, impact inflation and growth anticipations
                    #price indexes
                    'PPIFGS', 'PPIFCG', 'PPITM', 'PPICRM', 'OILPRICEX', 'PPICMM', 'NAPMPRI', # price indexes
                    'CPIAUCSL', #principal measure for inflation monitored by FED
                    'CPIAUCSL', 'CPIAPPSL', 'CPITRNSL','CPIMEDSL', 'CUSR0000SAC', 'CUUR0000SAD', 'CUSR0000SAS', 'CPIULFSL', 'CUUR0000SA0L2', 'CUSR0000SA0L5', #breakdown of the index for various sectors
                    'PCEPI', #inflation target for FED
                    'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA', #spendings for services, durable and nondurable goods
                    #stock markets
                    'S&P 500', 'S&P div yield', 'S&P PE ratio'
                    ]




We need other important variables to forecast the yield curve:
- the daily data of the yield curve itself 
- a measure of default risk of the US government, of its spending and budget. 

89